In [88]:
# Import Dependencies
import pandas as pd
import numpy as np

# Create reference to CSV file
fpath = "Resources/selfscoutdata.xlsx"
pd.reset_option("all")
pd.set_option('precision', 0)

formations = {

		'0 TRIPS':'EMPTY',
		'9 LOUIE NASTY':'EMPTY',
		'ACE':'2x2',
		'ALL':'UNBALANCED',
		'ALL KING':'UNBALANCED',
		'ALPHA':'3x1',
		'DEUCE':'2x2',
		'DOUBLES':'2x2',
		'DUO':'2x2',
		'EMPTY':'EMPTY',
		'EXIT':'EMPTY',
		'FLIP TRIPS':'3x1',
		'JOKER':'2x2',
		'JOKER SQUEEZE':'2x2',
		'KING':'3x1',
		'LOUIE':'3x1',
		'LOUIE FAR':'2x2',
		'LOUIE KING':'2x2',
		'LOUIE KING NASTY':'UNBALANCED',
		'LOUIE QUEEN':'3x1',
		'MINUTEMAN':'UNKNOWN',
		'QUADS':'EMPTY',
		'QUAKER':'UNKNOWN',
		'QUEEN':'2x2',
		'ROGER':'3x1',
		'ROGER FAR':'2x2',
		'ROGER KING':'3x1',
		'ROGER KING NASTY':'UNKNOWN',
		'ROGER QUEEN':'3x1',
		'SAINTS':'UNKNOWN',
		'SPLIT':'3x1',
		'TREY':'3x1',
		'TRIBE':'3x1',
		'TRIO':'3x1',
		'TRIPS':'3x1',
		'TROOP':'3x1',
		'TROY':'3x1',
		'TURBO':'UNKNOWN',
	}
# Import the CSV into a pandas DataFrame
raw = pd.read_excel(fpath)
raw["Name"]= raw["Name"].str.split(",", n = 1, expand = True)

# ADD TEAM FILTER OPTION HERE IF THERE IS ONE ********************************************************************
#raw = raw.loc[raw["Name"] == "20 08 UTSA OFF VS FAU DEF (10/31/2020)"]
# OTHERWISE WERE SHOWING OVERALL RP

raw.insert(2, "SPLITS", "na")
pers = raw

for index, row in raw.iterrows():
    raw.loc[index,'SPLITS'] = formations[row['FORM']]


: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



C:\Home\SkyKing\anaconda\lib\site-packages\pandas\_config\config.py:620: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [89]:
raw = raw[["FORM", "SPLITS", "R/P",]]

In [90]:
run = raw.loc[raw["R/P"] == "R"]
rp = run.groupby(["FORM", "SPLITS"])["R/P"].count().reset_index(name="R")
#rp

pas = raw.loc[raw["R/P"] == "P"]
pp = pas.groupby(["FORM", "SPLITS"])["R/P"].count().reset_index(name="P")

#rp.loc[rp["Name"] == "20 08 UTSA OFF VS FAU DEF (10/31/2020)"]

In [91]:
mer = rp.merge(pp, how='outer',on=["FORM","SPLITS"])
#mer = mer.set_index(["Name", "FORM", "SPLITS"])
#mer = rp.combine_first(pp)

mer.fillna(0,inplace=True)

In [92]:
fin = mer[["FORM", "SPLITS", "R", "P"]]
fin = fin.set_index(["FORM", "SPLITS"])

tt = (fin.div(fin.sum(axis=1,skipna = True), axis=0)*100).round(0).astype(str) + '%'
mer = fin.merge(tt, on=["FORM", "SPLITS"])

mer.rename(columns={'R_x': 'R', 'P_x': 'P', 'R_y': 'R%', 'P_y': 'P%'}, inplace=True)
mer = mer.reset_index()
mer.fillna(0,inplace=True)

In [93]:
############### summary table - group by splits based on above data ################
#mer.groupby(['SPLITS']).groups.keys()

sm = mer.groupby('SPLITS')['R','P'].sum()
rtot = sm['R'].sum()
ptot = sm['P'].sum()
sm = sm.reset_index()

df = pd.DataFrame({"SPLITS":['OVERALL'], "R":[rtot], "P":[ptot]}) 
df = df.reset_index()
mm = pd.concat([sm,df])
mm = mm[["SPLITS", "R", "P"]]
#mer[mer['Name'] == '20 08 UTSA OFF VS FAU DEF (10/31/2020)'].groupby('SPLITS')['R','P'].sum()

<ipython-input-93-3fbf14ad0aa7>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sm = mer.groupby('SPLITS')['R','P'].sum()


In [94]:
###### PERSONNEL BREAKDOWN ***********************************


In [95]:

run = pers[["OFF PERS","R/P",]]
run = run.loc[run["R/P"] == "R"]
run = run.groupby('OFF PERS')["R/P"].count().reset_index(name="R")

In [96]:
pas = pers[["OFF PERS","R/P",]]
pas = pas.loc[pas["R/P"] == "P"]
pas = pas.groupby('OFF PERS')["R/P"].count().reset_index(name="P")

In [97]:
pers = run.merge(pas,on="OFF PERS",how="outer")
pers.rename(columns={'OFF PERS': 'SPLITS'}, inplace=True)
pers.fillna(0,inplace=True)

In [98]:
total = pd.concat([pers,mm])

In [99]:
total = total.set_index(['SPLITS'])
cent = (total.div(total.sum(axis=1), axis=0)*100).round(0).astype(str) + '%'
total = total.merge(cent, on=['SPLITS'])
total.rename(columns={'R_x': 'R', 'P_x': 'P', 'R_y': 'R%', 'P_y': 'P%'}, inplace=True)
total = total.reset_index()
total

,SPLITS,R,P,R%,P%
0,11,196,286,41.0%,59.0%
1,12,66,46,59.0%,41.0%
2,10,0,5,0.0%,100.0%
3,2x2,81,158,34.0%,66.0%
4,3x1,124,139,47.0%,53.0%
5,EMPTY,4,11,27.0%,73.0%
6,UNBALANCED,15,2,88.0%,12.0%
7,UNKNOWN,38,27,58.0%,42.0%
8,OVERALL,262,337,44.0%,56.0%
